# Enseignement supérieur - round 2

Nous avons reçu un fichier plus complet et avons également amélioré l'outil de rapprochement. Nous faisons tourner cet outil sur le nouveau fichier.


In [ ]:
import csv
import json
import pandas as pd
from batid.services.guess_bdg_new import guess_all, report_format
from batid.utils.misc import is_float

raw_data_path = "data/bat_rnb_mesr_complet.csv"    
report_path = "results/results.json"


## Rapprochement

rows = []

with open(raw_data_path, "r") as file:
    reader = csv.DictReader(file, delimiter=";")

for row in reader:
    rows.append(_row_to_guess_params(row))

results = guess_all(rows)
report_data = report_format(results)

with open(self.report_path, "w") as file:
    json.dump(report_data, file)


In [ ]:
## Résultats

In [ ]:
with open(self.report_path, "r") as file:
    df = pd.json_normalize(json.load(file), sep="_")

# print total number of rows
total = len(df)
print(f"Total number of rows: {total}")

# How many have a match
matched = df[df["match_rnb_id"].notnull()]
matched_total = len(matched)
print(f"Total number of rows with a match: {matched_total}")

# List all possible values of matched_on_step and count rows for each
print(df["matched_on_step"].value_counts())